In [2]:
##librerias
import numpy as np
import sympy as sp
import math as mt
from math import pi,cos,sin


In [3]:
"medidas iniciales" 

codigo = "2172116"
x = [int(i) for i in codigo]
x = sum(x)


L = 0.2 * x #m
L_mm = L  #mm
l_cables = 1.5 #m 
l_cables_mm = l_cables * 1000 #mm
dia_perno = 1/2 * 25.4 #mm
dia_cables = 1/4 * 25.4 #mm6.35

f_s_ultimo = float(f"1.{x + 3}")
f_s_fluencia = float(f"1.{x}")
esf_fluencia_cortante  = 145 # Mpa
esf_ultimo_tension = 400     # Mpa
E = 200                      # Gpa

"angulos"
ang_beta = 40 * np.pi / 180 #radianes
ang_alfa = 50 * np.pi / 180 #redianes

"areas"
area_cable = np.pi * (dia_cables / 2) ** 2
area_pernos = np.pi * (dia_perno / 2)


In [4]:
"Equilibrio estatico externo"

Ax, Ay, FC, FD, FE, W = sp.symbols('Ax Ay FC FD FE W')

Fx = Ax - FC * sp.sin(ang_beta) + FE * sp.sin(ang_alfa)
eq_x = sp.Eq(Fx, 0)


Fy = Ay - W * L_mm + FC * sp.cos(ang_beta) * + FE * sp.cos(ang_alfa) + FD
eq_y = sp.Eq(Fy, 0)


M_a = - W * L_mm * 2 + FC * sp.cos(ang_beta) * L_mm + FE * sp.cos(ang_alfa) * L_mm + FD * L_mm
eq_Ma = sp.Eq(M_a, 0)


eq_x,  eq_y, eq_Ma


(Eq(Ax - 0.642787609686539*FC + 0.766044443118978*FE, 0),
 Eq(Ay + 0.492403876506104*FC*FE + FD - 4.0*W, 0),
 Eq(3.06417777247591*FC + 4.0*FD + 2.57115043874616*FE - 8.0*W, 0))

In [5]:
"ecuaciones de compatibilidad"
"ecuacion 1"
comp_1 = sp.Eq(FC - FD * sp.cos(ang_beta), 0)
comp_11 =  FD * sp.cos(ang_beta)
comp_1


Eq(FC - 0.766044443118978*FD, 0)

In [6]:
"ecuaciones de compatibilidad"
"ecuacion 2"
comp_2 = sp.Eq(FE - FD * sp.cos(ang_alfa), 0)
comp_22 =  FD * sp.cos(ang_alfa)
comp_2

   

Eq(-0.642787609686539*FD + FE, 0)

In [7]:
ecua = sp.Eq(M_a.subs([(FC, comp_11), (FE, comp_22)]), 0)
ecua




Eq(8.0*FD - 8.0*W, 0)

In [8]:
FD_W = sp.solve(ecua)[0][FD]
FD_W


W

In [9]:
"solucion de fuerzas en funcion de w"

FC_W = comp_11.subs(FD, FD_W)
FE_W = comp_22.subs(FD, FD_W)

print("la fuerza FC es igual a " + str(FC_W) + " Kn")
print("La fuerza FE es igual a " + str(FE_W) + " Kn")
print("EL valor de FD_w es igual a  " + str(FD_W) + " Kn") 

F_cables = [FC_W, FD_W, FE_W]

la fuerza FC es igual a 0.766044443118978*W Kn
La fuerza FE es igual a 0.642787609686539*W Kn
EL valor de FD_w es igual a  W Kn


In [10]:
"Calculo de la fuerza distribuida max para los cables y y pasador simples superiores"
W_max = []
FX2 = sp.symbols("FX2")
for F in F_cables:
    Esf_normal_cables = sp.Eq(FX2 / area_cable , esf_ultimo_tension * 10 ** 6 / f_s_ultimo)
    w_cab = sp.solve(Esf_normal_cables)[0] / 1000000000
    "cortante simple en pasadores superiores"
    esf_v_pasador_simple = sp.Eq(FX2 / area_pernos, esf_fluencia_cortante * 10 ** 6 / f_s_fluencia)
    w_pas = sp.solve( esf_v_pasador_simple)[0] / 10000000000
    print(w_cab, w_pas)
    W_max.append(w_cab)
    W_max.append(w_pas)                                   
                              
    
print("la carga w max de los cables [Kn / m]") 
min(W_max)
print()



10.2989325019816 0.241051786316066
10.2989325019816 0.241051786316066
10.2989325019816 0.241051786316066
la carga w max de los cables [Kn / m]



In [11]:
"Cortante en el apoyo A"
FvA = sp.sqrt(Ay ** 2  +  Ax ** 2)
FvA

sqrt(Ax**2 + Ay**2)

In [12]:
"Punto de conexión de los cables en B"
Fvx_doble = comp_22.subs(FD, FD_W) * sp.sin(ang_alfa) - comp_11.subs(FD, FD_W) * sp.sin(ang_beta)
Fvy_doble = comp_22.subs(FD, FD_W) * sp.cos(ang_alfa) + FD_W + comp_11.subs(FD, FD_W) * sp.cos(ang_beta) 
FvB_doble = sp.sqrt(Fvx_doble ** 2 + Fvy_doble ** 2)
FvB_doble 


2.0*sqrt(W**2)

In [13]:
"calculo de Wmax teniendo encuenta los cortantes simples y dobles en los pernos "
F_apoyos = [FvA, FvB_doble]
w_max_apoyos = []
for f in F_apoyos:
    Equ_esf_cortante = sp.Eq(F / area_pernos, esf_fluencia_cortante * 10 ** 6 / f_s_fluencia)
    w_pasadores = sp.solve(esf_v_pasador_simple)[0] / 1000
    w_max_apoyos.append(w_pas)

print("Carga distribuida máxima por pasadores [kN / m]")
carga_max = min(W_max)
carga_max

Carga distribuida máxima por pasadores [kN / m]


0.241051786316066

In [14]:
print("Carga distribuida máxima por pasadores [kN / m]")
carga_max = min(W_max)
carga_max
print(carga_max)

"CÁLCULO DE FUERZAS Y DEFORMACIONES"
Fuerza_E = comp_11.subs([(FD, FD_W), (W, carga_max)])
deformación_E = (Fuerza_E * l_cables_mm) / (area_cable * E * 10 ** 9)
## Fuerza en kilonewtons  (kN) y deformación en (mm)
Fuerza_E, deformación_E * 1000
print("Deformación en BE:", deformación_E)
print("Fuerza de BE:", Fuerza_E)


Fuerza_C = comp_22.subs([(FD, FD_W), (W, carga_max)])
deformación_C = (Fuerza_C * 1000 * l_cables) / (area_cable * E * 10 ** 9)
## Fuerza en kilonewtons  (kN) y deformación en (mm)
Fuerza_C, deformación_C * 1000
print("Deformación en BC:", deformación_C)
print("Fuerza de BC:", Fuerza_C)


Fuerza_D = FD_W.subs(W, carga_max)
deformación_D = (Fuerza_D * 1000 * l_cables) / (area_cable * E * 10 ** 9)
Fuerza_D, deformación_D * 1000
## Fuerza en kilonewtons  (kN) y deformación en (mm)
print("Deformación en BD:",deformación_D)
print("Fuerza en BD", Fuerza_D)


Fuerza_Ax = Fx.subs([(FE, Fuerza_E), (FC, Fuerza_C)])
Fuerza_Ax = sp.solve(sp.Eq(Fuerza_Ax, 0)) [0]
## Fuerza en kilonewtons  (kN) 
Fuerza_Ax
print("Reacción Ax:", Fuerza_Ax)


Fuerza_Ay = Fy.subs([(FE, Fuerza_E), (FC, Fuerza_C), (FD, Fuerza_D), (W, carga_max)])
Fuerza_Ay = sp.solve(sp.Eq(Fuerza_Ay, 0)) [0]
## Fuerza en kilonewtons  (kN) 
Fuerza_Ay
print("Reacción Ay:", Fuerza_Ay)

Carga distribuida máxima por pasadores [kN / m]
0.241051786316066
Deformación en BE: 4.37308835638786e-11
Fuerza de BE: 0.184656381411326
Deformación en BC: 3.66945682695071e-11
Fuerza de BC: 0.154945101536775
Deformación en BD: 5.70866141732282e-11
Fuerza en BD 0.241051786316066
Reacción Ax: -0.0418582034171432
Reacción Ay: 0.709066895324985
